# ChatterCheetah Data Analysis

This notebook provides comprehensive data analysis for the ChatterCheetah platform.

**Designed for:** Vertex AI Workbench (GCP)

## What This Notebook Covers

1. **Tenant Overview** - Active tenants and their usage
2. **Conversation Analytics** - Volume, channels, timing patterns
3. **Message Analysis** - Response patterns, content metrics
4. **Lead Capture Performance** - Conversion rates, contact info quality
5. **Channel Analysis** - Web vs SMS vs Voice performance

## Setup

Before running, ensure:
1. Environment variables are set (same as main app)
2. Database connectivity is available (Cloud SQL via private networking)
3. Required packages are installed: `uv add pandas matplotlib seaborn`

---
## 1. Environment Setup

In [ ]:
# Standard library
import os
import sys
from datetime import datetime, timedelta
from collections import defaultdict

# Add project root to path for imports
project_root = os.path.dirname(os.path.abspath(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Data analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Async support for notebooks
import nest_asyncio
nest_asyncio.apply()

import asyncio

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
sns.set_palette('husl')

print("✓ Imports complete")

In [ ]:
# Database imports
from sqlalchemy import text, select, func
from sqlalchemy.ext.asyncio import AsyncSession

# App imports
from notebooks.utils import get_db_session, setup_tenant_context
from app.persistence.models.tenant import Tenant, User
from app.persistence.models.conversation import Conversation, Message
from app.persistence.models.lead import Lead
from app.persistence.database import AsyncSessionLocal

print("✓ Database modules loaded")

In [ ]:
# Helper functions for async database queries

async def fetch_all_tenants() -> pd.DataFrame:
    """Fetch all tenants as DataFrame."""
    async with get_db_session() as session:
        result = await session.execute(select(Tenant))
        tenants = result.scalars().all()
        return pd.DataFrame([
            {
                'id': t.id,
                'name': t.name,
                'subdomain': t.subdomain,
                'is_active': t.is_active,
                'created_at': t.created_at
            }
            for t in tenants
        ])

async def fetch_all_conversations() -> pd.DataFrame:
    """Fetch all conversations as DataFrame."""
    async with get_db_session() as session:
        result = await session.execute(select(Conversation))
        conversations = result.scalars().all()
        return pd.DataFrame([
            {
                'id': c.id,
                'tenant_id': c.tenant_id,
                'channel': c.channel,
                'external_id': c.external_id,
                'created_at': c.created_at,
                'updated_at': c.updated_at
            }
            for c in conversations
        ])

async def fetch_all_messages() -> pd.DataFrame:
    """Fetch all messages as DataFrame."""
    async with get_db_session() as session:
        result = await session.execute(select(Message))
        messages = result.scalars().all()
        return pd.DataFrame([
            {
                'id': m.id,
                'conversation_id': m.conversation_id,
                'role': m.role,
                'content': m.content,
                'content_length': len(m.content) if m.content else 0,
                'sequence_number': m.sequence_number,
                'created_at': m.created_at
            }
            for m in messages
        ])

async def fetch_all_leads() -> pd.DataFrame:
    """Fetch all leads as DataFrame."""
    async with get_db_session() as session:
        result = await session.execute(select(Lead))
        leads = result.scalars().all()
        return pd.DataFrame([
            {
                'id': l.id,
                'tenant_id': l.tenant_id,
                'conversation_id': l.conversation_id,
                'email': l.email,
                'phone': l.phone,
                'name': l.name,
                'extra_data': l.extra_data,
                'has_email': l.email is not None,
                'has_phone': l.phone is not None,
                'has_name': l.name is not None,
                'created_at': l.created_at
            }
            for l in leads
        ])

def run_async(coro):
    """Helper to run async coroutines in notebook."""
    return asyncio.get_event_loop().run_until_complete(coro)

print("✓ Helper functions defined")

---
## 2. Load Data

In [ ]:
# Load all data from database
print("Loading data from database...")

df_tenants = run_async(fetch_all_tenants())
df_conversations = run_async(fetch_all_conversations())
df_messages = run_async(fetch_all_messages())
df_leads = run_async(fetch_all_leads())

print(f"\n✓ Data loaded:")
print(f"  - Tenants: {len(df_tenants):,}")
print(f"  - Conversations: {len(df_conversations):,}")
print(f"  - Messages: {len(df_messages):,}")
print(f"  - Leads: {len(df_leads):,}")

---
## 3. Tenant Overview

In [ ]:
# Tenant Summary Statistics
if len(df_tenants) > 0:
    print("=" * 50)
    print("TENANT SUMMARY")
    print("=" * 50)
    print(f"Total Tenants: {len(df_tenants)}")
    print(f"Active Tenants: {df_tenants['is_active'].sum()}")
    print(f"Inactive Tenants: {(~df_tenants['is_active']).sum()}")
    print()
    
    # Join with conversation counts
    if len(df_conversations) > 0:
        conv_counts = df_conversations.groupby('tenant_id').size().reset_index(name='conversation_count')
        lead_counts = df_leads.groupby('tenant_id').size().reset_index(name='lead_count') if len(df_leads) > 0 else pd.DataFrame({'tenant_id': [], 'lead_count': []})
        
        tenant_stats = df_tenants.merge(conv_counts, left_on='id', right_on='tenant_id', how='left')
        tenant_stats = tenant_stats.merge(lead_counts, left_on='id', right_on='tenant_id', how='left', suffixes=('', '_lead'))
        tenant_stats['conversation_count'] = tenant_stats['conversation_count'].fillna(0).astype(int)
        tenant_stats['lead_count'] = tenant_stats['lead_count'].fillna(0).astype(int)
        
        display_cols = ['name', 'subdomain', 'is_active', 'conversation_count', 'lead_count', 'created_at']
        print(tenant_stats[display_cols].to_string(index=False))
else:
    print("No tenants found in database.")

In [ ]:
# Tenant Activity Over Time
if len(df_conversations) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Conversations per tenant
    conv_by_tenant = df_conversations.groupby('tenant_id').size().sort_values(ascending=True)
    if len(conv_by_tenant) > 0:
        tenant_names = df_tenants.set_index('id')['name'].to_dict()
        conv_by_tenant.index = conv_by_tenant.index.map(lambda x: tenant_names.get(x, f'Tenant {x}'))
        conv_by_tenant.plot(kind='barh', ax=axes[0], color='steelblue')
        axes[0].set_xlabel('Number of Conversations')
        axes[0].set_ylabel('Tenant')
        axes[0].set_title('Conversations by Tenant')
    
    # Conversations over time
    df_conversations['date'] = pd.to_datetime(df_conversations['created_at']).dt.date
    daily_convs = df_conversations.groupby('date').size()
    if len(daily_convs) > 0:
        daily_convs.plot(ax=axes[1], color='green', linewidth=2)
        axes[1].set_xlabel('Date')
        axes[1].set_ylabel('Conversations')
        axes[1].set_title('Daily Conversation Volume')
        axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("No conversations to visualize.")

---
## 4. Conversation Analytics

In [ ]:
# Conversation Statistics
if len(df_conversations) > 0:
    print("=" * 50)
    print("CONVERSATION SUMMARY")
    print("=" * 50)
    print(f"Total Conversations: {len(df_conversations):,}")
    
    # Channel breakdown
    print("\nBy Channel:")
    channel_counts = df_conversations['channel'].value_counts()
    for channel, count in channel_counts.items():
        pct = (count / len(df_conversations)) * 100
        print(f"  {channel}: {count:,} ({pct:.1f}%)")
    
    # Time-based metrics
    if 'created_at' in df_conversations.columns and len(df_conversations) > 0:
        df_conversations['hour'] = pd.to_datetime(df_conversations['created_at']).dt.hour
        df_conversations['day_of_week'] = pd.to_datetime(df_conversations['created_at']).dt.day_name()
        
        print("\nBusiest Hours (top 5):")
        hour_counts = df_conversations['hour'].value_counts().head(5)
        for hour, count in hour_counts.items():
            print(f"  {hour:02d}:00 - {count:,} conversations")
        
        print("\nBusiest Days:")
        day_counts = df_conversations['day_of_week'].value_counts()
        for day, count in day_counts.items():
            print(f"  {day}: {count:,}")
else:
    print("No conversations found.")

In [ ]:
# Conversation Visualizations
if len(df_conversations) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Channel distribution
    channel_counts = df_conversations['channel'].value_counts()
    colors = sns.color_palette('husl', len(channel_counts))
    axes[0, 0].pie(channel_counts.values, labels=channel_counts.index, autopct='%1.1f%%', colors=colors)
    axes[0, 0].set_title('Conversations by Channel')
    
    # Hourly distribution
    if 'hour' in df_conversations.columns:
        hourly = df_conversations['hour'].value_counts().sort_index()
        axes[0, 1].bar(hourly.index, hourly.values, color='teal', alpha=0.7)
        axes[0, 1].set_xlabel('Hour of Day')
        axes[0, 1].set_ylabel('Conversations')
        axes[0, 1].set_title('Conversations by Hour')
        axes[0, 1].set_xticks(range(0, 24, 2))
    
    # Day of week distribution
    if 'day_of_week' in df_conversations.columns:
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        day_counts = df_conversations['day_of_week'].value_counts().reindex(day_order).fillna(0)
        axes[1, 0].bar(day_counts.index, day_counts.values, color='coral', alpha=0.7)
        axes[1, 0].set_xlabel('Day of Week')
        axes[1, 0].set_ylabel('Conversations')
        axes[1, 0].set_title('Conversations by Day of Week')
        axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Conversation duration (updated_at - created_at)
    df_conversations['duration_minutes'] = (
        pd.to_datetime(df_conversations['updated_at']) - 
        pd.to_datetime(df_conversations['created_at'])
    ).dt.total_seconds() / 60
    
    # Filter reasonable durations (< 1 day)
    reasonable_durations = df_conversations[df_conversations['duration_minutes'] < 1440]['duration_minutes']
    if len(reasonable_durations) > 0:
        axes[1, 1].hist(reasonable_durations, bins=30, color='purple', alpha=0.7, edgecolor='black')
        axes[1, 1].set_xlabel('Duration (minutes)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Conversation Duration Distribution')
        axes[1, 1].axvline(reasonable_durations.median(), color='red', linestyle='--', label=f'Median: {reasonable_durations.median():.1f} min')
        axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("No conversations to visualize.")

---
## 5. Message Analysis

In [ ]:
# Message Statistics
if len(df_messages) > 0:
    print("=" * 50)
    print("MESSAGE SUMMARY")
    print("=" * 50)
    print(f"Total Messages: {len(df_messages):,}")
    
    # Role breakdown
    print("\nBy Role:")
    role_counts = df_messages['role'].value_counts()
    for role, count in role_counts.items():
        pct = (count / len(df_messages)) * 100
        print(f"  {role}: {count:,} ({pct:.1f}%)")
    
    # Messages per conversation
    msgs_per_conv = df_messages.groupby('conversation_id').size()
    print(f"\nMessages per Conversation:")
    print(f"  Average: {msgs_per_conv.mean():.1f}")
    print(f"  Median: {msgs_per_conv.median():.1f}")
    print(f"  Min: {msgs_per_conv.min()}")
    print(f"  Max: {msgs_per_conv.max()}")
    
    # Content length stats
    print(f"\nContent Length (characters):")
    for role in df_messages['role'].unique():
        role_msgs = df_messages[df_messages['role'] == role]
        print(f"  {role}:")
        print(f"    Average: {role_msgs['content_length'].mean():.0f}")
        print(f"    Median: {role_msgs['content_length'].median():.0f}")
else:
    print("No messages found.")

In [ ]:
# Message Visualizations
if len(df_messages) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Role distribution
    role_counts = df_messages['role'].value_counts()
    colors = {'user': '#3498db', 'assistant': '#2ecc71', 'system': '#9b59b6'}
    role_colors = [colors.get(r, 'gray') for r in role_counts.index]
    axes[0, 0].bar(role_counts.index, role_counts.values, color=role_colors)
    axes[0, 0].set_xlabel('Role')
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].set_title('Messages by Role')
    
    # Messages per conversation histogram
    msgs_per_conv = df_messages.groupby('conversation_id').size()
    axes[0, 1].hist(msgs_per_conv, bins=20, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0, 1].set_xlabel('Messages per Conversation')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Distribution of Conversation Length')
    axes[0, 1].axvline(msgs_per_conv.mean(), color='red', linestyle='--', label=f'Mean: {msgs_per_conv.mean():.1f}')
    axes[0, 1].legend()
    
    # Content length by role
    df_messages.boxplot(column='content_length', by='role', ax=axes[1, 0])
    axes[1, 0].set_xlabel('Role')
    axes[1, 0].set_ylabel('Content Length (characters)')
    axes[1, 0].set_title('Message Length by Role')
    plt.suptitle('')  # Remove automatic title
    
    # Response time analysis (time between messages)
    df_messages_sorted = df_messages.sort_values(['conversation_id', 'sequence_number'])
    df_messages_sorted['prev_created_at'] = df_messages_sorted.groupby('conversation_id')['created_at'].shift(1)
    df_messages_sorted['response_time_seconds'] = (
        pd.to_datetime(df_messages_sorted['created_at']) - 
        pd.to_datetime(df_messages_sorted['prev_created_at'])
    ).dt.total_seconds()
    
    # Filter assistant responses (response to user)
    assistant_responses = df_messages_sorted[
        (df_messages_sorted['role'] == 'assistant') & 
        (df_messages_sorted['response_time_seconds'] > 0) &
        (df_messages_sorted['response_time_seconds'] < 60)  # Under 1 minute
    ]
    
    if len(assistant_responses) > 0:
        axes[1, 1].hist(assistant_responses['response_time_seconds'], bins=30, color='green', alpha=0.7, edgecolor='black')
        axes[1, 1].set_xlabel('Response Time (seconds)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('AI Response Time Distribution')
        axes[1, 1].axvline(assistant_responses['response_time_seconds'].median(), color='red', linestyle='--', 
                          label=f'Median: {assistant_responses["response_time_seconds"].median():.2f}s')
        axes[1, 1].legend()
    else:
        axes[1, 1].text(0.5, 0.5, 'Insufficient response time data', ha='center', va='center', transform=axes[1, 1].transAxes)
    
    plt.tight_layout()
    plt.show()
else:
    print("No messages to visualize.")

---
## 6. Lead Capture Analysis

In [ ]:
# Lead Statistics
if len(df_leads) > 0:
    print("=" * 50)
    print("LEAD CAPTURE SUMMARY")
    print("=" * 50)
    print(f"Total Leads: {len(df_leads):,}")
    
    # Conversion rate (leads with conversations)
    if len(df_conversations) > 0:
        leads_with_conv = df_leads[df_leads['conversation_id'].notna()]
        print(f"Leads with Conversations: {len(leads_with_conv):,}")
        conv_rate = (len(df_leads) / len(df_conversations)) * 100 if len(df_conversations) > 0 else 0
        print(f"Lead Capture Rate: {conv_rate:.1f}%")
    
    # Contact info quality
    print("\nContact Information Quality:")
    print(f"  Has Email: {df_leads['has_email'].sum():,} ({(df_leads['has_email'].sum()/len(df_leads))*100:.1f}%)")
    print(f"  Has Phone: {df_leads['has_phone'].sum():,} ({(df_leads['has_phone'].sum()/len(df_leads))*100:.1f}%)")
    print(f"  Has Name: {df_leads['has_name'].sum():,} ({(df_leads['has_name'].sum()/len(df_leads))*100:.1f}%)")
    
    # Complete leads (all info)
    complete_leads = df_leads[df_leads['has_email'] & df_leads['has_phone'] & df_leads['has_name']]
    print(f"  Complete (all fields): {len(complete_leads):,} ({(len(complete_leads)/len(df_leads))*100:.1f}%)")
    
    # Extra data analysis (Zapier integration)
    leads_with_extra = df_leads[df_leads['extra_data'].notna()]
    print(f"\nLeads with Extra Data (Zapier): {len(leads_with_extra):,}")
    
    # Leads by tenant
    print("\nLeads by Tenant:")
    lead_by_tenant = df_leads.groupby('tenant_id').size().sort_values(ascending=False)
    tenant_names = df_tenants.set_index('id')['name'].to_dict()
    for tenant_id, count in lead_by_tenant.items():
        tenant_name = tenant_names.get(tenant_id, f'Tenant {tenant_id}')
        print(f"  {tenant_name}: {count:,}")
else:
    print("No leads found.")
    
    if len(df_conversations) > 0:
        print(f"\n(Total conversations: {len(df_conversations):,} - no leads captured yet)")

In [ ]:
# Lead Visualizations
if len(df_leads) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Contact info breakdown
    contact_data = {
        'Email': df_leads['has_email'].sum(),
        'Phone': df_leads['has_phone'].sum(),
        'Name': df_leads['has_name'].sum()
    }
    axes[0, 0].bar(contact_data.keys(), contact_data.values(), color=['#3498db', '#2ecc71', '#e74c3c'])
    axes[0, 0].set_ylabel('Count')
    axes[0, 0].set_title('Lead Contact Information Availability')
    axes[0, 0].axhline(len(df_leads), color='gray', linestyle='--', label=f'Total Leads: {len(df_leads)}')
    axes[0, 0].legend()
    
    # Leads over time
    df_leads['date'] = pd.to_datetime(df_leads['created_at']).dt.date
    daily_leads = df_leads.groupby('date').size()
    if len(daily_leads) > 0:
        daily_leads.plot(ax=axes[0, 1], color='orange', linewidth=2, marker='o', markersize=4)
        axes[0, 1].set_xlabel('Date')
        axes[0, 1].set_ylabel('Leads')
        axes[0, 1].set_title('Daily Lead Capture')
        axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Leads by tenant
    lead_by_tenant = df_leads.groupby('tenant_id').size().sort_values(ascending=True)
    tenant_names = df_tenants.set_index('id')['name'].to_dict()
    lead_by_tenant.index = lead_by_tenant.index.map(lambda x: tenant_names.get(x, f'Tenant {x}'))
    lead_by_tenant.plot(kind='barh', ax=axes[1, 0], color='purple')
    axes[1, 0].set_xlabel('Number of Leads')
    axes[1, 0].set_ylabel('Tenant')
    axes[1, 0].set_title('Leads by Tenant')
    
    # Lead quality distribution
    df_leads['quality_score'] = df_leads['has_email'].astype(int) + df_leads['has_phone'].astype(int) + df_leads['has_name'].astype(int)
    quality_dist = df_leads['quality_score'].value_counts().sort_index()
    quality_labels = {0: 'None', 1: 'One field', 2: 'Two fields', 3: 'Complete'}
    quality_dist.index = quality_dist.index.map(lambda x: quality_labels.get(x, str(x)))
    axes[1, 1].bar(quality_dist.index, quality_dist.values, color=['#e74c3c', '#f39c12', '#3498db', '#2ecc71'][:len(quality_dist)])
    axes[1, 1].set_xlabel('Lead Quality')
    axes[1, 1].set_ylabel('Count')
    axes[1, 1].set_title('Lead Quality Distribution')
    
    plt.tight_layout()
    plt.show()
else:
    print("No leads to visualize.")

---
## 7. Channel Performance Analysis

In [ ]:
# Channel Performance Comparison
if len(df_conversations) > 0 and len(df_messages) > 0:
    print("=" * 50)
    print("CHANNEL PERFORMANCE")
    print("=" * 50)
    
    # Merge conversations with messages
    conv_msg_counts = df_messages.groupby('conversation_id').size().reset_index(name='msg_count')
    conv_analysis = df_conversations.merge(conv_msg_counts, left_on='id', right_on='conversation_id', how='left')
    conv_analysis['msg_count'] = conv_analysis['msg_count'].fillna(0)
    
    # Stats by channel
    for channel in conv_analysis['channel'].unique():
        channel_data = conv_analysis[conv_analysis['channel'] == channel]
        print(f"\n{channel.upper()}:")
        print(f"  Conversations: {len(channel_data):,}")
        print(f"  Avg Messages/Conv: {channel_data['msg_count'].mean():.1f}")
        
        # Lead conversion for channel
        if len(df_leads) > 0:
            channel_conv_ids = set(channel_data['id'])
            channel_leads = df_leads[df_leads['conversation_id'].isin(channel_conv_ids)]
            lead_rate = (len(channel_leads) / len(channel_data)) * 100 if len(channel_data) > 0 else 0
            print(f"  Leads: {len(channel_leads):,}")
            print(f"  Lead Capture Rate: {lead_rate:.1f}%")
        
        # Duration stats
        if 'duration_minutes' in channel_data.columns:
            print(f"  Avg Duration: {channel_data['duration_minutes'].mean():.1f} min")
else:
    print("Insufficient data for channel analysis.")

In [ ]:
# Channel Comparison Charts
if len(df_conversations) > 0 and len(df_messages) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Merge for analysis
    conv_msg_counts = df_messages.groupby('conversation_id').size().reset_index(name='msg_count')
    conv_analysis = df_conversations.merge(conv_msg_counts, left_on='id', right_on='conversation_id', how='left')
    
    # Messages per conversation by channel
    channel_msg_avg = conv_analysis.groupby('channel')['msg_count'].mean()
    axes[0].bar(channel_msg_avg.index, channel_msg_avg.values, color=sns.color_palette('husl', len(channel_msg_avg)))
    axes[0].set_xlabel('Channel')
    axes[0].set_ylabel('Avg Messages per Conversation')
    axes[0].set_title('Engagement by Channel')
    
    # Duration by channel
    if 'duration_minutes' in conv_analysis.columns:
        channel_duration = conv_analysis.groupby('channel')['duration_minutes'].mean()
        axes[1].bar(channel_duration.index, channel_duration.values, color=sns.color_palette('husl', len(channel_duration)))
        axes[1].set_xlabel('Channel')
        axes[1].set_ylabel('Avg Duration (minutes)')
        axes[1].set_title('Conversation Duration by Channel')
    
    # Lead capture rate by channel
    if len(df_leads) > 0:
        lead_rates = {}
        for channel in conv_analysis['channel'].unique():
            channel_convs = conv_analysis[conv_analysis['channel'] == channel]
            channel_conv_ids = set(channel_convs['id'])
            channel_leads = df_leads[df_leads['conversation_id'].isin(channel_conv_ids)]
            lead_rates[channel] = (len(channel_leads) / len(channel_convs)) * 100 if len(channel_convs) > 0 else 0
        
        axes[2].bar(lead_rates.keys(), lead_rates.values(), color=sns.color_palette('husl', len(lead_rates)))
        axes[2].set_xlabel('Channel')
        axes[2].set_ylabel('Lead Capture Rate (%)')
        axes[2].set_title('Lead Conversion by Channel')
    else:
        axes[2].text(0.5, 0.5, 'No lead data available', ha='center', va='center', transform=axes[2].transAxes)
    
    plt.tight_layout()
    plt.show()
else:
    print("Insufficient data for channel visualization.")

---
## 8. Export Data for External Analysis

In [ ]:
# Export data to CSV files for external analysis (e.g., BigQuery, Looker, Zapier)
import os
from datetime import datetime

export_dir = 'exports'
os.makedirs(export_dir, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Export each dataset
exports = {
    'tenants': df_tenants,
    'conversations': df_conversations,
    'messages': df_messages,
    'leads': df_leads
}

print("Exporting data to CSV...")
for name, df in exports.items():
    if len(df) > 0:
        filepath = f"{export_dir}/{name}_{timestamp}.csv"
        df.to_csv(filepath, index=False)
        print(f"  ✓ {name}: {filepath} ({len(df):,} rows)")
    else:
        print(f"  - {name}: (empty, skipped)")

print(f"\nExport complete. Files saved to: {export_dir}/")

---
## 9. Summary Report

In [ ]:
# Generate Summary Report
print("=" * 60)
print("           CHATTERCHEETAH DATA ANALYSIS SUMMARY")
print("=" * 60)
print(f"Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# Platform Overview
print("📊 PLATFORM OVERVIEW")
print("-" * 40)
print(f"  Active Tenants: {df_tenants['is_active'].sum() if len(df_tenants) > 0 else 0}")
print(f"  Total Conversations: {len(df_conversations):,}")
print(f"  Total Messages: {len(df_messages):,}")
print(f"  Total Leads: {len(df_leads):,}")
print()

# Engagement Metrics
if len(df_messages) > 0 and len(df_conversations) > 0:
    print("💬 ENGAGEMENT METRICS")
    print("-" * 40)
    msgs_per_conv = df_messages.groupby('conversation_id').size()
    print(f"  Avg Messages per Conversation: {msgs_per_conv.mean():.1f}")
    if 'duration_minutes' in df_conversations.columns:
        print(f"  Avg Conversation Duration: {df_conversations['duration_minutes'].median():.1f} min")
    print()

# Lead Performance
if len(df_leads) > 0:
    print("🎯 LEAD PERFORMANCE")
    print("-" * 40)
    capture_rate = (len(df_leads) / len(df_conversations)) * 100 if len(df_conversations) > 0 else 0
    print(f"  Lead Capture Rate: {capture_rate:.1f}%")
    complete_leads = df_leads[df_leads['has_email'] & df_leads['has_phone'] & df_leads['has_name']]
    quality_rate = (len(complete_leads) / len(df_leads)) * 100
    print(f"  Complete Lead Rate: {quality_rate:.1f}%")
    print()

# Channel Breakdown
if len(df_conversations) > 0:
    print("📱 CHANNEL BREAKDOWN")
    print("-" * 40)
    for channel, count in df_conversations['channel'].value_counts().items():
        pct = (count / len(df_conversations)) * 100
        print(f"  {channel}: {count:,} ({pct:.1f}%)")
    print()

print("=" * 60)
print("End of Report")

---
## 10. Custom Query Section

Use this section to run custom SQL queries or pandas operations on the loaded data.

In [ ]:
# Example: Custom SQL Query
# Uncomment and modify as needed

# async def run_custom_query(query: str) -> pd.DataFrame:
#     """Run a custom SQL query and return as DataFrame."""
#     async with get_db_session() as session:
#         result = await session.execute(text(query))
#         rows = result.fetchall()
#         columns = result.keys()
#         return pd.DataFrame(rows, columns=columns)

# # Example query
# query = """
# SELECT 
#     t.name as tenant_name,
#     COUNT(DISTINCT c.id) as conversation_count,
#     COUNT(m.id) as message_count
# FROM tenants t
# LEFT JOIN conversations c ON t.id = c.tenant_id
# LEFT JOIN messages m ON c.id = m.conversation_id
# GROUP BY t.id, t.name
# ORDER BY conversation_count DESC
# """
# 
# df_custom = run_async(run_custom_query(query))
# display(df_custom)

print("Custom query section - uncomment and modify the code above to run custom queries.")

In [ ]:
# Quick data exploration commands
# Uncomment as needed

# View first few rows of any dataset
# df_conversations.head(10)
# df_messages.head(10)
# df_leads.head(10)

# Get column info
# df_conversations.info()

# Describe numeric columns
# df_messages.describe()

# Filter examples
# df_conversations[df_conversations['channel'] == 'web']
# df_leads[df_leads['has_email'] == True]

print("Data exploration section - uncomment lines above to explore the loaded data.")